In [1]:
%load_ext autoreload

In [2]:
%matplotlib nbagg

In [3]:
%load_ext line_profiler

In [ ]:
import time

import numpy as np

import xspcontrol as xc
import pyxsp as px

In [ ]:
s = px.System('/opt/xsp/config/system.yml')
if s is None:
    raise RuntimeError('X-Spectrum system creation failed, aborting.')
s.connect()
s.initialize()
d = s.open_detector('lambda')
r = s.open_receiver('lambda/1')

In [ ]:
s.list_receivers()

In [ ]:
d.number_of_frames

In [ ]:
while not r.ram_allocated:
    time.sleep(0.1)
while not d.voltage_settled(1):
    time.sleep(0.1)

In [ ]:
d.number_of_frames = 2
d.start_acquisition()
print("frame_depth", r.frame_depth)
try:
    for i in range(11):
        f = r.get_frame()
        try:
            print("nr", f.nr)
            print("subframe", f.subframe)
            print("connector", f.connector)
            print("status_code", f.status_code)
            print("data", type(f.data), f.data)
            print("seq", f.seq)
            print("trigger", f.trigger)
        finally:
            r.release_frame(f)
finally:
    d.stop_acquisition()

In [ ]:
d.number_of_frames = 1
d.start_acquisition()
# print(r.frame_depth)

f = r.get_frame()
print("nr", f.nr)
print("subframe", f.subframe)
print("connector", f.connector)
print("status_code", f.status_code)
print("data", type(f.data), f.data)
print("seq", f.seq)
print("trigger", f.trigger)


In [ ]:
r.release_frame(f)
d.stop_acquisition()

In [ ]:
r.frame_depth

In [7]:
%autoreload
from libertem_live.api import LiveContext
from libertem_live.detectors.xspectrum import XSpectrumAcquisition
from libertem_live.detectors.xspectrum.acquisition import XSpectrumCommHandler
from libertem_live.udf.monitor import SignalMonitorUDF
from libertem.viz.bqp import BQLive2DPlot


In [8]:
%autoreload
ctx = LiveContext(plot_class=BQLive2DPlot)

fatal: not a git repository: /code/LiberTEM/../../../../../.git/modules/LiberTEM
fatal: not a git repository: /code/LiberTEM/../../../../../.git/modules/LiberTEM
fatal: not a git repository: /code/LiberTEM/../../../../../.git/modules/LiberTEM
fatal: not a git repository: /code/LiberTEM/../../../../../.git/modules/LiberTEM


In [ ]:
%autoreload
aq = XSpectrumAcquisition(nav_shape=(128, 4), trigger=lambda aq: None)

In [ ]:
%autoreload
aq = aq.initialize(ctx.executor)

In [ ]:
ctx.run_udf(dataset=aq, udf=SignalMonitorUDF(), plots=True, progress=True)

In [ ]:
# lprun -f XSpectrumCommHandler.handle_task ctx.run_udf(dataset=aq, udf=SignalMonitorUDF(), plots=True, progress=True)

In [20]:
from unittest import mock
from contextlib import contextmanager
import numpy as np

In [ ]:
@contextmanager
def connect():
    import pyxsp as px

    s = px.System('/opt/xsp/config/system.yml')
    if s is None:
        raise RuntimeError('X-Spectrum system creation failed, aborting.')
    try:
        s.connect()
        s.initialize()

        d = tuple(s.open_detector(id) for id in s.list_detectors())
        r = tuple(s.open_receiver(id) for id in s.list_receivers())
        yield DetectorConnection(
            system=s,
            detectors=d,
            receivers=r,
        )
    finally:
        pass

In [ ]:
import sys
from typing import NamedTuple 

def test():
    import pyxsp as px
    print('test', px)

with mock.patch.dict(sys.modules, {'pyxsp': mock.MagicMock()}):
    data = np.random.randint(0, 23, (23, 42, 74))
    

        
    test()
    
    class MockDetectorReceiver(mock.MagicMixin):
        '''
        An instance of this mocks the detector and
        the receiver simultaneously since that makes
        management of settings, counters etc easier.
        '''
        def __init__(self, data):
            assert len(data.shape) == 3
            self._data = data
            self._counter = 0
            self._seq = 0
            self.number_of_frames = 1
            self.frame_depth = data.itemsize * 8
        
        def get_frame(self, timeout=None):
            if self._counter >= self.number_of_frames:
                return None
            index = self._counter % len(self._data)
            result_data = self._data[index].flatten()
            
            result = mock.MagicMock()
            result.seq = self._seq
            result.nr = self._counter
            result.data = memoryview(result_data)
            
            self._seq += 1
            self._counter += 1
            return result
        
        
        @property
        def frame_height(self):
            return self._data.shape[1]
        
        @property
        def frame_height(self):
            return self._data.shape[1]
        
        def start_acquisition(self):
            self.counter = 0
    
    class DetectorConnection(NamedTuple):
        system: object
        detectors: object
        receivers: object
    
    
    mock_detectorreceiver = MockDetectorReceiver(data)
    
    import pyxsp as px
    
    print(px)
    mock_system = mock.MagicMock()
    mock_system.list_detectors = mock.MagicMock(return_value=('lambda',))
    mock_system.list_receivers = mock.MagicMock(return_value=('lambda/1',))
    
    mock_system.open_detector = mock.MagicMock(return_value=mock_detectorreceiver)
    mock_system.open_receiver = mock.MagicMock(return_value=mock_detectorreceiver)
    
    px.System = mock.MagicMock(return_value=mock_system)
    
    with connect() as connection:

        s = px.System()
        print(s, s.list_detectors())
        print(connection)
        

In [10]:
from libertem_live.detectors.xspectrum.mock import mock_xspectrum
import numpy as np

In [21]:
%autoreload
data = np.random.randint(0, 23, (23, 42, 51))
with mock_xspectrum(data):
    trigger = mock.MagicMock(return_value=None)
    aq = XSpectrumAcquisition(nav_shape=(128, 4), trigger=trigger)
    aq = aq.initialize(ctx.executor)
    ctx.run_udf(dataset=aq, udf=SignalMonitorUDF(), plots=True, progress=True)
    
    trigger.assert_called_with(aq)

Figure(axes=[Axis(label='x', scale=LinearScale(max=1.0, min=0.0)), Axis(label='y', orientation='vertical', sca…

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:01<00:00,  2.12it/s]
